# Santander Customer Satisfaction

## Планы

* Hyperopt для xgboost

## Решение

### Общее начало

In [54]:
%matplotlib inline

import os
import copy
import random
import pickle
from importlib import reload

import numpy as np
import scipy as sp
import pandas as pd
import pylab

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cross_validation import StratifiedKFold, cross_val_score

import xgboost as xgb

import ml_training.common as common
import ml_training.vwutils as vwutils
import sklensemble.ensemblers.averagers as averagers

In [ ]:
reload(common)

In [62]:
reload(vwutils)

<module 'ml_training.vwutils' from '/home/andrey/projects/ml_training/vwutils.py'>

In [ ]:
reload(averagers)

In [3]:
def find_constant_columns(df):
    return [column for column in df.columns if len(df[column].unique()) < 2]

In [4]:
def find_equal_columns(df):
    equal_columns = []
    for col_num, column1 in enumerate(df.columns[: -1]):
        for column2 in df.columns[col_num+1:]:
            if not (df[column1] - df[column2]).any():
                equal_columns.append(column2)
    return list(set(equal_columns))

In [5]:
RANDOM_STATE = 100
random.seed(RANDOM_STATE)

In [6]:
train_df = pd.read_csv(os.path.join("input", "train.csv"))

In [7]:
constant_columns = find_constant_columns(train_df)
equal_columns = find_equal_columns(train_df)
drop_columns = list(set(constant_columns).union(equal_columns))

In [8]:
train_df.drop(drop_columns, inplace=True, axis=1)

In [9]:
train_x = train_df.loc[:, "var3": "var38"]
train_y = train_df["TARGET"]

In [10]:
if_mask = common.get_independent_features_mask(train_x.values, tol=5e-7, is_plot=False) # 5e-7
uncorr_mask = common.find_corr_features_mask(train_x.values, trashhold=0.995) # 0.995
res_mask = if_mask & uncorr_mask
if_mask.sum(), uncorr_mask.sum(), res_mask.sum()

(257, 255, 224)

In [11]:
train_x_if = train_x.loc[:, if_mask]
train_x_uc = train_x.loc[:, uncorr_mask]

In [12]:
cv = StratifiedKFold(train_y, 5, shuffle=True, random_state=RANDOM_STATE)

### Создание классификаторов

In [13]:
xgbc = xgb.XGBClassifier(n_estimators=570, max_depth=5, seed=1234, colsample_bytree=0.7, learning_rate=0.02,
                         subsample=0.68)

In [14]:
xgbc_weighted = xgb.XGBClassifier(n_estimators=570, max_depth=5, seed=1234, colsample_bytree=0.7, learning_rate=0.02,
                                  subsample=0.68)

In [15]:
xgbc_if = xgb.XGBClassifier(n_estimators=570, max_depth=5, seed=1234, colsample_bytree=0.7, learning_rate=0.02,
                            subsample=0.68)

In [16]:
xgbc_uc = xgb.XGBClassifier(n_estimators=570, max_depth=5, seed=1234, colsample_bytree=0.7, learning_rate=0.02,
                            subsample=0.68)

In [17]:
estimators = [copy.deepcopy(xgbc), copy.deepcopy(xgbc_weighted), copy.deepcopy(xgbc_if), copy.deepcopy(xgbc_uc)]
is_weighted = [False, True, False, False]
x_mask = [None, None, if_mask, uncorr_mask]
bc = averagers.BlendingClassifier(estimators, is_weighted, x_mask)

### Grid search

In [ ]:
params = {"n_estimators": [150], "max_depth": [3], "seed": [RANDOM_STATE]}
params = {"n_estimators": [570], "max_depth": [5], "seed": [1234], "learning_rate": [0.02], "subsample": [0.68],
          "colsample_bytree": [0.7]}
gs = GridSearchCV(xgbc, params, scoring="roc_auc", cv=cv, n_jobs=common.NCPU, refit=False, verbose=10)
gs.fit(train_x, train_y)

### Веса

In [18]:
w0 = 1
w1 = train_y.value_counts()[0] / train_y.value_counts()[1] * 0.5
weights = np.array(list((map(lambda x: w1 if x else w0, train_y))))

### Обучение классификаторов

In [ ]:
xgbc.fit(train_x.values, train_y.values)

In [ ]:
xgbc_weighted.fit(train_x.values, train_y.values, weights)

In [ ]:
xgbc_if.fit(train_x_if.values, train_y.values)

In [ ]:
xgbc_uc.fit(train_x_uc.values, train_y.values)

In [ ]:
bc.fit(train_x.values, train_y.values, weights)

### Выходная оценка

In [19]:
test_df = pd.read_csv(os.path.join("input", "test.csv"))
test_df.drop(drop_columns, inplace=True, axis=1)
test_x = test_df.loc[:, "var3": "var38"]

In [20]:
test_y = xgbc_very_hard.predict_proba(test_x)[:, 1]

NameError: name 'xgbc_very_hard' is not defined

In [ ]:
common.save_output(np.array([test_df["ID"].values, test_y]).T, ["ID", "TARGET"])

### Тестирование классификаторов

In [ ]:
cross_val_score(copy.deepcopy(xgbc), train_x, train_y, scoring="roc_auc", cv=cv, n_jobs=common.NCPU, verbose=1)

In [ ]:
cross_val_score(copy.deepcopy(xgbc), train_x, train_y, scoring="roc_auc", cv=cv, n_jobs=common.NCPU, verbose=1)

In [ ]:
common.cross_val_score_with_weights(copy.deepcopy(bc), train_x.values, train_y.values, weights, "roc_auc", cv)

In [ ]:
np.corrcoef(np.vstack([xgbc.predict_proba(test_x.values)[:, 1], xgbc_weighted.predict_proba(test_x.values)[:, 1],
                       xgbc_if.predict_proba(test_x.values)[:, 1], xgbc_uc.predict_proba(test_x.values)[:, 1],
                       bc.predict_proba(test_x.values)[:, 1]]))

In [ ]:
precision_recall_fscore_support(train_y.values, xgbc.predict(train_x.values))

In [ ]:
precision_recall_fscore_support(train_y.values, xgbc_weighted.predict(train_x.values))

In [ ]:
precision_recall_fscore_support(train_y.values, xgbc_if.predict(train_x.values))

In [ ]:
precision_recall_fscore_support(train_y.values, xgbc_uc.predict(train_x.values))

In [ ]:
precision_recall_fscore_support(train_y.values, bc.predict(train_x.values))

In [ ]:
pylab.figure(figsize=(16, 6))

pylab.grid(True)
pylab.scatter(xgbc.predict_proba(train_x.values)[:, 1], bc.predict_proba(train_x.values)[:, 1], alpha=0.5, color = 'red')

### VW

In [61]:
input_train_file = os.path.join("temp", "scs.input_train")
vwutils.make_vw_input(train_x, input_train_file, y=train_y)

{'num_op_var40_ult3': 99, 'num_venta_var44_hace3': 261, 'ind_var13_largo': 33, 'num_trasp_var17_out_ult1': 257, 'num_var39_0': 119, 'num_var4': 69, 'num_op_var40_ult1': 98, 'ind_var17_0': 38, 'num_med_var22_ult3': 222, 'saldo_medio_var13_corto_hace3': 280, 'saldo_medio_var33_hace2': 297, 'num_op_var39_efect_ult1': 245, 'num_var30_0': 108, 'saldo_medio_var13_corto_ult3': 282, 'num_aport_var17_hace3': 209, 'num_meses_var5_ult3': 224, 'ind_var37_0': 60, 'saldo_medio_var44_ult1': 303, 'saldo_var12': 131, 'num_trasp_var17_in_hace3': 255, 'imp_op_var40_comer_ult1': 5, 'ind_var6_0': 23, 'saldo_medio_var5_ult3': 270, 'num_var6_0': 72, 'num_var31': 111, 'saldo_medio_var12_hace2': 275, 'imp_op_var40_efect_ult3': 8, 'num_meses_var13_corto_ult3': 227, 'imp_op_var40_comer_ult3': 6, 'saldo_medio_var17_hace3': 290, 'num_var42_0': 123, 'num_op_var41_efect_ult3': 244, 'num_trasp_var33_out_ult1': 260, 'ind_var24': 45, 'num_var33_0': 113, 'saldo_medio_var29_hace2': 293, 'num_op_var40_comer_ult1': 237, 'n

In [63]:
input_test_file = os.path.join("temp", "scs.input_test")
vwutils.make_vw_input(test_x, input_test_file)

### Тестирование кода

In [ ]:
x = train_x.iloc[: 1000].values
y = train_y.iloc[: 1000].values
w = weights[: 1000]
x_t = test_x.iloc[: 1000].values

In [43]:
pred_df = pd.read_csv(os.path.join("temp", "scs.pred"), header=None)

In [51]:
pred_df[0].value_counts()

-1    75818
dtype: int64